In [ ]:
# This is an AI Agent to bakwards engineer contact information for varios public github repos that are shown to use certain dev tools and are qualified as potential sales targets.
# This Agent also performs developer mapping and creates a profile based on a devs previously used toolset, 


In [ ]:
Get List of Public Repos that are relevant. Scrape GitHub, figure out rate limiting. Get list of every app using open ai. Add x to a vector db

To switch search, do x

In [41]:
Public Repo:
    
How to tell if it's openAI's free tier, or would be paid:

use LLM to get all contributors
Analyze Name & Email in commits
Check the email address in commit metadata, (if it's not anonymized by GitHub)
username, name, bio, email, company, location, blog, followers, repos, contributions
                                             
Tools: git log or use GitHub's web interface to see commit history.
Github Profile Bio
                                             
                                             
Here is how chatGPT scraped Ashar's Identity: https://chatgpt.com/c/67e4a0ef-8d90-8008-8d3d-fafb4c1bd3a2                                                                         
People often include their real name, location, personal website or even Linkedin. Scrape Bio, Find their name from that.                                         
Combine GitHub name/location info with job titles to find likely LinkedIn profiles.

Tools: LinkedIn Sales Navigator, PhantomBuster, or custom OpenAI-based fuzzy matche
                                             
                                             
Reasoning:
                                             
                                             
🥈 Semi-Reliable: Cross-Referencing
Gravatar Lookup
GitHub avatars come from Gravatar, which uses email hashes. You can sometimes reverse the hash if you have a suspected email.
Handle Matching
Look for the same handle across platforms (e.g., GitHub, LinkedIn, Twitter, Dev.to, Stack Overflow).
Tools: Sherlock, Namechk
Repository Context
People sometimes name repos after their employers, universities, or personal projects (e.g., my-thesis-harvard).
Contributors to Known Projects
If they contribute to a company repo, check the org's team page.                                             
For the Devs you can't get in touch with - make a PR                                             
                                             
                                             
                                             
private repo mapping

                                             
List of Companies you know are working on AI projects do Mapping project with every source you can find.


Dev Mapping

Most interesting repos are private. Need to figure out what tooling the devs have typically used, make an educated mapping profile on all of 


#Identify Repos that are using x foundational model over another foundational model. Do this for all foundational models, and then frameworks
#Filter by commits, number of people, filter by whatever else to find projects that are actually relevant. 



                                             



SyntaxError: invalid syntax (3560752671.py, line 4)

In [10]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [11]:
import csv
import os
from dotenv import load_dotenv, dotenv_values
load_dotenv()

import requests
import time
import pandas as pd
import json
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")


In [58]:
headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": GITHUB_TOKEN
}


#MAKE LIST OF TOOLS YOU WANT TO BUILD A PROFILE FOR
# if you want langchain OR claude, put OR in. If there is nothing, the space acts like an 'and'
query = "langchain claude chroma in:file language:python"
url = f"https://api.github.com/search/code?q={query}&per_page=5&page=1"
              
response = requests.get(url, headers=headers)
data = response.json()

if response.status_code != 200:
    print("Error:", data.get("message", "Unknown error"))
    print("Status code:", response.status_code)
    print("Response headers:", response.headers)
else:
   for item in data.get("items", []):
        repo = item["repository"]
        print("📦 Repo Name:", repo["full_name"])
        print("🔗 Repo URL: ", repo["html_url"])
        print("Contributors", repo["contributors_url"])
        print("-" * 40)


#print(json.dumps(data, indent=2))

📦 Repo Name: OptimalScale/LMFlow
🔗 Repo URL:  https://github.com/OptimalScale/LMFlow
Contributors https://api.github.com/repos/OptimalScale/LMFlow/contributors
----------------------------------------
📦 Repo Name: h2oai/h2ogpt
🔗 Repo URL:  https://github.com/h2oai/h2ogpt
Contributors https://api.github.com/repos/h2oai/h2ogpt/contributors
----------------------------------------
📦 Repo Name: shlomota/MishnahBot
🔗 Repo URL:  https://github.com/shlomota/MishnahBot
Contributors https://api.github.com/repos/shlomota/MishnahBot/contributors
----------------------------------------
📦 Repo Name: alfredcs/cavatar
🔗 Repo URL:  https://github.com/alfredcs/cavatar
Contributors https://api.github.com/repos/alfredcs/cavatar/contributors
----------------------------------------
📦 Repo Name: filip-michalsky/SalesGPT
🔗 Repo URL:  https://github.com/filip-michalsky/SalesGPT
Contributors https://api.github.com/repos/filip-michalsky/SalesGPT/contributors
----------------------------------------


In [60]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"{GITHUB_TOKEN}"
}

query = "langchain claude chroma in:file language:python"
url = f"https://api.github.com/search/code?q={query}&per_page=5&page=1"

response = requests.get(url, headers=headers)
data = response.json()

contributors_data = []

if response.status_code != 200:
    print("Error:", data.get("message", "Unknown error"))
    print("Status code:", response.status_code)
else:
    seen_repos = set()  # Avoid duplicate repos from code search

    for item in data.get("items", []):
        repo = item["repository"]
        full_name = repo["full_name"]

        if full_name in seen_repos:
            continue
        seen_repos.add(full_name)

        repo_name = repo["full_name"]
        repo_url = repo["html_url"]
        contributors_url = repo["contributors_url"]

        print("📦 Repo Name:", repo_name)
        print("🔗 Repo URL:", repo_url)

        contrib_response = requests.get(contributors_url, headers=headers)
        if contrib_response.status_code == 200:
            contributors = contrib_response.json()
            contributor_list = [
                {
                    "login": c["login"],
                    "contributions": c["contributions"],
                    "profile_url": c["html_url"]
                }
                for c in contributors[:5]  # limit to top 5
            ]
        else:
            contributor_list = []
            print("❌ Failed to fetch contributors:", contrib_response.status_code)

        contributors_data.append({
            "repo": repo_name,
            "repo_url": repo_url,
            "contributors": contributor_list
        })

        print("👥 Contributors:")
        for c in contributor_list:
            print(f"  - {c['login']} ({c['contributions']}) — {c['profile_url']}")
        print("-" * 50)

# ✅ Now contributors_data is ready for LLM, CSV, etc.

import csv

csv_file = "contributors.csv"

# Define the CSV column headers
fieldnames = ["repo", "repo_url", "contributor", "contributions", "profile_url"]

with open(csv_file, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    for entry in contributors_data:
        for c in entry["contributors"]:
            writer.writerow({
                "repo": entry["repo"],
                "repo_url": entry["repo_url"],
                "contributor": c["login"],
                "contributions": c["contributions"],
                "profile_url": c["profile_url"]
            })

print(f"✅ Saved contributor data to {csv_file}")


📦 Repo Name: OptimalScale/LMFlow
🔗 Repo URL: https://github.com/OptimalScale/LMFlow
👥 Contributors:
  - research4pan (440) — https://github.com/research4pan
  - wheresmyhair (298) — https://github.com/wheresmyhair
  - shizhediao (181) — https://github.com/shizhediao
  - hendrydong (176) — https://github.com/hendrydong
  - yaoguany (88) — https://github.com/yaoguany
--------------------------------------------------
📦 Repo Name: h2oai/h2ogpt
🔗 Repo URL: https://github.com/h2oai/h2ogpt
👥 Contributors:
  - pseudotensor (6451) — https://github.com/pseudotensor
  - arnocandel (831) — https://github.com/arnocandel
  - achraf-mer (173) — https://github.com/achraf-mer
  - tloen (84) — https://github.com/tloen
  - fatihozturkh2o (60) — https://github.com/fatihozturkh2o
--------------------------------------------------
📦 Repo Name: shlomota/MishnahBot
🔗 Repo URL: https://github.com/shlomota/MishnahBot
👥 Contributors:
  - shlomota (10) — https://github.com/shlomota
------------------------------

In [61]:
print(contributors_data)

[{'repo': 'OptimalScale/LMFlow', 'repo_url': 'https://github.com/OptimalScale/LMFlow', 'contributors': [{'login': 'research4pan', 'contributions': 440, 'profile_url': 'https://github.com/research4pan'}, {'login': 'wheresmyhair', 'contributions': 298, 'profile_url': 'https://github.com/wheresmyhair'}, {'login': 'shizhediao', 'contributions': 181, 'profile_url': 'https://github.com/shizhediao'}, {'login': 'hendrydong', 'contributions': 176, 'profile_url': 'https://github.com/hendrydong'}, {'login': 'yaoguany', 'contributions': 88, 'profile_url': 'https://github.com/yaoguany'}]}, {'repo': 'h2oai/h2ogpt', 'repo_url': 'https://github.com/h2oai/h2ogpt', 'contributors': [{'login': 'pseudotensor', 'contributions': 6451, 'profile_url': 'https://github.com/pseudotensor'}, {'login': 'arnocandel', 'contributions': 831, 'profile_url': 'https://github.com/arnocandel'}, {'login': 'achraf-mer', 'contributions': 173, 'profile_url': 'https://github.com/achraf-mer'}, {'login': 'tloen', 'contributions': 8

In [63]:
contributor_data = []
for entry in contributors_data:
    repo = entry["repo"]
    repo_url = entry["repo_url"]

    for c in entry["contributors"]:
        # Fetch full user profile
        user_resp = requests.get(f"https://api.github.com/users/{c['login']}", headers=headers)
        if user_resp.status_code != 200:
            print(f"⚠️ Failed to get details for {c['login']}")
            continue
        user_data = user_resp.json()

        contributor_data.append({
            "repo": repo,
            "repo_url": repo_url,
            "login": c["login"],
            "contributions": c["contributions"],
            "profile_url": c["profile_url"],
            "name": user_data.get("name"),
            "company": user_data.get("company"),
            "blog": user_data.get("blog"),
            "location": user_data.get("location"),
            "email": user_data.get("email"),
            "bio": user_data.get("bio"),
            "twitter_username": user_data.get("twitter_username"),
            "followers": user_data.get("followers"),
            "public_repos": user_data.get("public_repos")
        })

        
        import csv

fields = [
    "repo", "repo_url",
    "login", "contributions", "profile_url",
    "name", "company", "blog", "location", "email",
    "bio", "twitter_username", "followers", "public_repos"
]

with open("all_contributors.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    for row in contributor_data:
        writer.writerow(row)

print("✅ Full contributor data saved to all_contributors.csv")


✅ Full contributor data saved to all_contributors.csv


In [ ]:
#LLM Prompt:
#is this code for business purposes? Is someone paying for this? In various scenarios, how much is this costing them. Realistically, why are they using the x tool, and why would x make sense?



In [ ]:

add to a folder
get requirements, dockerfile to better extract code?

Get LLM to backfind contact info for each contributor and to make a profile


In [18]:
# https://chatgpt.com/c/67e2fa1a-2850-8008-99a8-bd908a6eb335


In [2]:
# 🔍 1. Scrape and Analyze Public Repositories (e.g., GitHub)
# What to look for:

# ML frameworks: TensorFlow, PyTorch, Hugging Face Transformers, etc.

# Infrastructure: Docker, Kubernetes, Terraform

# Languages: Python, Java, Go, etc.

# Tooling: MLflow, Weights & Biases, Ray, etc.

# How to do it:

# Use GitHub’s API to pull repo metadata, requirements.txt, Dockerfile, etc.

# Parse for keywords and dependencies.

# Analyze commit history and contributors to see which teams or engineers worked on what.

SyntaxError: invalid character '🔍' (U+1F50D) (3781455242.py, line 1)

In [ ]:
# 🧠 2. Use Job Postings for Stack Clues
# What to look for:

# Job listings for ML Engineers, MLOps, or Applied Scientists.

# Stacks and tools mentioned (they’re usually very specific).

# Tools to help:

# Diffbot or Apollo.io for scraping and structuring job post data.

# Simple web scraping + LLM pipeline to extract and summarize

In [ ]:
# 🌐 3. Explore Company Engineering Blogs & Tech Talks
# Places to check:

# Medium, Dev.to, Substack, company tech blogs.

# YouTube talks or conference slides (e.g., NeurIPS, PyData, MLConf).

# How to analyze:

# Use an agent (e.g. LangChain + LLM) to summarize or cross-reference stacks across companies.

In [ ]:
# 🧩 4. LinkedIn Engineering Profiles
# Use case:

# Search for engineers in ML roles at the target companies.

# Analyze their LinkedIn summaries and endorsements for tooling or stack mentions.

# Bonus: You can build a mini-graph of tools used across the org based on who lists what.

In [ ]:
# 📦 5. Inspect Their Products (Client/Server Side)
# For web-based ML products:

# Use browser dev tools or tools like Wappalyzer to detect front-end and API tools.

# Look for WebAssembly, ONNX.js, TensorFlow.js, etc.

# For APIs:

# If their ML product exposes APIs, use Postman or curl to inspect endpoints and infer architecture.

In [ ]:
# 🧪 6. Use Prebuilt Tools & Databases
# Examples:

# StackShare.io – Check stack disclosures by company.

# Crunchbase – Look at ML acquisitions, product descriptions.

# BuiltWith – Web tech stack detection.

# Papers With Code – For more research-y orgs, see what papers are tied to which models and codebases.

In [ ]:
# 🧠 7. Build an Agentic AI Researcher
# If you're aiming for scale, consider building an agent that:

# Takes in a list of companies.

# Searches GitHub, job sites, blogs, LinkedIn.

# Extracts stack mentions.

# Aggregates and ranks the tools based on relevance or frequency.